In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/CS550/SRCNN/'

/content/drive/MyDrive/CS550/SRCNN


In [ ]:
pwd

'/content/drive/.shortcut-targets-by-id/1oE8U0pU-coSldw8OMDE7dzHH0664tLkj/CS550/SRCNN'

In [ ]:
import os
import math
import sys

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
from scipy.ndimage.filters import gaussian_filter

from PIL import Image
from skimage.metrics import structural_similarity as ssim

import torch
import torch.nn as nn
import torch.nn.functional as F

from helper.distort_images import distort_image
from helper.utils import ycbcr2rgb
from helper.utils import psnr
from helper.evaluate_models import evaluate_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SR_FACTOR = 3

train_pixel_mean = 0.43411854967058106
train_pixel_std = 0.23247274722791464

In [ ]:
# !ffmpeg -i input.avi -r 50/1 data/build/input/%03d.bmp

In [ ]:
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=9, padding=4);
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0);
        self.conv3 = nn.Conv2d(32, 1, kernel_size=7, padding=3);

    def forward(self, img):
        out = F.relu(self.conv1(img)) #n_1=64 feature maps
        out = F.relu(self.conv2(out)) #non linear mapping
        out = self.conv3(out)
        return out
print(torch.cuda.is_available())

True


In [ ]:
SR_model = SRCNN().to(device)
SR_model.load_state_dict(torch.load('./checkpoint/test_checkpoint_SRCNN_w_MSE.pt' , map_location='cpu')['model_state_dict'])

<All keys matched successfully>

In [ ]:
SR_model

SRCNN(
  (conv1): Conv2d(1, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (conv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(32, 1, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
)

In [ ]:
pwd

'/content/drive/My Drive/CS550/SRCNN'

In [ ]:
SR_model.conv1.padding = (4,4)
SR_model.conv3.padding = (3,3)
test5_img_names = [im for im in os.listdir('/content/drive/MyDrive/CS550/GANFingerprints-master/ProGAN/gen/progan_original') if im[-4:] == '.png']

# print(test5_img_names[0])
for test_im in test5_img_names:
    # print(test_im)
    path = '/content/drive/MyDrive/CS550/GANFingerprints-master/ProGAN/gen/progan_original/' + test_im
    blurred_test_im = distort_image(path=path, factor=SR_FACTOR, sigma=1, blur=True)
    ImageFile = Image.open(path)
    im = np.array(ImageFile.convert('YCbCr')) 
    

    im_Y, im_Cb, im_Cr = im[:, :, 0], im[:, :, 1], im[:, :, 2]
    im_Y = (im_Y.astype(np.int16)).astype(np.int64)
    width, length = im_Y.shape
    im = Image.fromarray(im, mode='YCbCr')
    im = im.resize(size=(length*3, width*3))
    im = np.array(im.convert('YCbCr'))
    #normalize
    input = im[:,:,0] / 255.0
    #standardize
    input -= train_pixel_mean
    input /= train_pixel_std
    im_out_Y = SR_model(torch.tensor(input, dtype=torch.float).unsqueeze(0).unsqueeze(0).to(device))
    im_out_Y = im_out_Y.detach().squeeze().squeeze().cpu().numpy().astype(np.float)
    im_out_viz = np.zeros((im_out_Y.shape[0], im_out_Y.shape[1], 3))
    
    #unstandardize 
    im_out_Y = (im_out_Y * train_pixel_std) + train_pixel_mean
    
    #un-normalize
    im_out_Y *= 255.0

    # print(im_out_Y.shape)
    # print(im_out_viz.shape)
    # print(im.shape)
    # print(im[:,:,1])
    # print(im[:,:,2])
    im_out_viz[:,:,0] = im_out_Y
    im_out_viz[:,:,1] = im[:,:,1]
    im_out_viz[:,:,2] = im[:,:,2]

    img = ycbcr2rgb(im_out_viz)
    img = Image.fromarray(img, 'RGB')
    name =  '/content/drive/MyDrive/CS550/GANFingerprints-master/ProGAN/gen/progan_sr/' + test_im[:-4] + '_sr_3x' + '.png'
    print(name)
    img.save(name)


In [ ]:
# !ffmpeg -r 50 -i ./data/build/output/%03d.png -vcodec libx264 -y -an output.avi -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2"

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
# !rm ./data/build/input/*

In [ ]:
# !rm ./data/build/output/*

In [ ]:
SR_model.conv1.padding = (4,4)
SR_model.conv3.padding = (3,3)
test5_img_names = [im for im in sorted(os.listdir('/content/drive/MyDrive/CS550/GANFingerprints-master/ProGAN/gen/progan_original')) if im[-4:] == '.png'][:4]

# print(test5_img_names[0])
for test_im in test5_img_names:
    # print(test_im)
    path = '/content/drive/MyDrive/CS550/GANFingerprints-master/ProGAN/gen/progan_original/' + test_im
    blurred_test_im = distort_image(path=path, factor=SR_FACTOR, sigma=1, blur=True)
    ImageFile = Image.open(path)
    im = np.array(ImageFile.convert('YCbCr')) 
    

    im_Y, im_Cb, im_Cr = im[:, :, 0], im[:, :, 1], im[:, :, 2]
    im_Y = (im_Y.astype(np.int16)).astype(np.int64)
    width, length = im_Y.shape
    im = Image.fromarray(im, mode='YCbCr')
    # im = im.resize(size=(length*3, width*3))
    im = np.array(im.convert('YCbCr'))
    #normalize
    input = im[:,:,0] / 255.0
    #standardize
    input -= train_pixel_mean
    input /= train_pixel_std
    im_out_Y = SR_model(torch.tensor(input, dtype=torch.float).unsqueeze(0).unsqueeze(0).to(device))
    im_out_Y = im_out_Y.detach().squeeze().squeeze().cpu().numpy().astype(np.float)
    im_out_viz = np.zeros((im_out_Y.shape[0], im_out_Y.shape[1], 3))
    
    #unstandardize 
    im_out_Y = (im_out_Y * train_pixel_std) + train_pixel_mean
    
    #un-normalize
    im_out_Y *= 255.0

    # print(im_out_Y.shape)
    # print(im_out_viz.shape)
    # print(im.shape)
    # print(im[:,:,1])
    # print(im[:,:,2])
    im_out_viz[:,:,0] = im_out_Y
    im_out_viz[:,:,1] = im_Cb
    im_out_viz[:,:,2] = im_Cr

    img = ycbcr2rgb(im_out_viz)
    img = Image.fromarray(img, 'RGB')
    name =  '/content/drive/MyDrive/CS550/GANFingerprints-master/ProGAN/gen/progan_sr/' + test_im[:-4] + '_sr_3x' + '.png'
    print(name)
    img.save(name)


/content/drive/MyDrive/CS550/GANFingerprints-master/ProGAN/gen/progan_sr/ProGAN_00000000_sr_3x.png
/content/drive/MyDrive/CS550/GANFingerprints-master/ProGAN/gen/progan_sr/ProGAN_00000001_sr_3x.png
/content/drive/MyDrive/CS550/GANFingerprints-master/ProGAN/gen/progan_sr/ProGAN_00000002_sr_3x.png
/content/drive/MyDrive/CS550/GANFingerprints-master/ProGAN/gen/progan_sr/ProGAN_00000003_sr_3x.png
